In [1]:
spark

Waiting for a Spark session to start...

Spark application_1540458187951_76729: Some(http://gw02.itversity.com:4044)

In [2]:
sc.getConf.getAll.filter(_._2.contains("/proxy/"))(0)._2

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1540458187951_76729

In [3]:
def getType(o: Any) = o.getClass.getCanonicalName

getType: (o: Any)String


In [4]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getenv("HOME").split("/")(2)

os_name = Linux
hdfs_home = /user/kranthidr


/user/kranthidr

In [5]:
val path = hdfs_home +"/dataSets/spark-guide/binary-classification"

path = /user/kranthidr/dataSets/spark-guide/binary-classification


/user/kranthidr/dataSets/spark-guide/binary-classification

In [6]:
// in Scala
val bInput = spark.read.format("parquet").load(path)
  .selectExpr("features", "cast(label as double) as label")

bInput = [features: vector, label: double]


[features: vector, label: double]

In [7]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.classification.LogisticRegression
val lr = new LogisticRegression()
println(lr.explainParams()) // see all parameters
val lrModel = lr.fit(bInput)

aggregationDepth: suggested depth for treeAggregate (>= 2) (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial. (default: auto)
featuresCol: features column name (default: features)
fitIntercept: whether to fit an intercept term (default: true)
labelCol: label column name (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. (undefined)
maxIter: maximum number of iterations (>= 0) (default: 100)
predictionCol: prediction column name (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: 

lr = logreg_7a90b6c11129
lrModel = logreg_7a90b6c11129


logreg_7a90b6c11129

In [8]:
// COMMAND ----------

// in Scala
println(lrModel.coefficients)
println(lrModel.intercept)

[6.848741326854929,0.3535658901019745,14.814900276915923]
-10.22569586448109


In [9]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.classification.BinaryLogisticRegressionSummary
val summary = lrModel.summary
val bSummary = summary.asInstanceOf[BinaryLogisticRegressionSummary]
println(bSummary.areaUnderROC)
bSummary.roc.show()
bSummary.pr.show()

1.0
+---+------------------+
|FPR|               TPR|
+---+------------------+
|0.0|               0.0|
|0.0|0.3333333333333333|
|0.0|               1.0|
|1.0|               1.0|
|1.0|               1.0|
+---+------------------+

+------------------+---------+
|            recall|precision|
+------------------+---------+
|               0.0|      1.0|
|0.3333333333333333|      1.0|
|               1.0|      1.0|
|               1.0|      0.6|
+------------------+---------+



summary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@13717962
bSummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@13717962


org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@13717962

In [10]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.classification.DecisionTreeClassifier
val dt = new DecisionTreeClassifier()
println(dt.explainParams())
val dtModel = dt.fit(bInput)

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. (default: false)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext (default: 10)
featuresCol: features column name (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (default: gini)
labelCol: label column name (default: label)
maxBins: Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 5)
maxMe

dt = dtc_7a0a5fa078a1
dtModel = DecisionTreeClassificationModel (uid=dtc_7a0a5fa078a1) of depth 1 with 3 nodes


DecisionTreeClassificationModel (uid=dtc_7a0a5fa078a1) of depth 1 with 3 nodes

In [11]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.classification.RandomForestClassifier
val rfClassifier = new RandomForestClassifier()
println(rfClassifier.explainParams())
val trainedModel = rfClassifier.fit(bInput)

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. (default: false)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]. (default: auto)
featuresCol: features column name (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (default: gini)
labelCol: label column name (default: label)
maxBins: Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any ca

rfClassifier = rfc_e28b1e9e88e3
trainedModel = RandomForestClassificationModel (uid=rfc_e28b1e9e88e3) with 20 trees


RandomForestClassificationModel (uid=rfc_e28b1e9e88e3) with 20 trees

In [12]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.classification.GBTClassifier
val gbtClassifier = new GBTClassifier()
println(gbtClassifier.explainParams())
val trainedModel = gbtClassifier.fit(bInput)

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. (default: false)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]. (default: all)
featuresCol: features column name (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (default: gini)
labelCol: label column name (default: label)
lossType: Loss function which GBT tries to minimize (case-insensitive). Supported options: logistic (default: logistic

gbtClassifier = gbtc_dd57fecdd750
trainedModel = GBTClassificationModel (uid=gbtc_dd57fecdd750) with 20 trees


GBTClassificationModel (uid=gbtc_dd57fecdd750) with 20 trees

In [13]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.classification.NaiveBayes
val nb = new NaiveBayes()
println(nb.explainParams())
val trainedModel = nb.fit(bInput.where("label != 0"))

featuresCol: features column name (default: features)
labelCol: label column name (default: label)
modelType: The model type which is a string (case-sensitive). Supported options: multinomial (default) and bernoulli. (default: multinomial)
predictionCol: prediction column name (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name (default: rawPrediction)
smoothing: The smoothing parameter. (default: 1.0)
thresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0 excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that clas

nb = nb_e9619defc147
trainedModel = NaiveBayesModel (uid=nb_e9619defc147) with 1 classes


NaiveBayesModel (uid=nb_e9619defc147) with 1 classes

In [14]:
// COMMAND ----------

// in Scala
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

val out = lrModel.transform(bInput)
  .select("prediction", "label")
  .rdd.map(x => (x(0).asInstanceOf[Double], x(1).asInstanceOf[Double]))

val metrics = new BinaryClassificationMetrics(out)

out = MapPartitionsRDD[494] at map at <console>:49
metrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@26a738ff


org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@26a738ff

In [15]:
// COMMAND ----------

// in Scala
metrics.areaUnderPR
metrics.areaUnderROC
println("Receiver Operating Characteristic")
metrics.roc.toDF().show()


// COMMAND ----------

Receiver Operating Characteristic
+---+---+
| _1| _2|
+---+---+
|0.0|0.0|
|0.0|1.0|
|1.0|1.0|
|1.0|1.0|
+---+---+

